### Indicadores demográficos a nivel Ciudad de México

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, stringr, writexl, foreign, downloader, tools, archive)

Cargando paquete requerido: pacman



Descargar los datos de Defunciones Registradas de INEGI

In [2]:
#Definiciones para antes de descargar los datos
dir.create('Estadísticas de Defunciones Registradas (EDR)')
# Crear directorio principal
dir_datos <- 'Estadísticas de Defunciones Registradas (EDR)/Microdatos_1'
if (!dir.exists(dir_datos)) {
  dir.create(dir_datos, recursive = TRUE)
}
print(file.exists(dir_datos)) #True = directorio creado

Warning message in dir.create("Estadísticas de Defunciones Registradas (EDR)"):
"'Estadísticas de Defunciones Registradas (EDR)' already exists"


[1] TRUE


In [25]:
descargar_defunciones_dbf <- function() {
  options(timeout = 90000)
  lista_defun <- list()
  
  # Lista de URLs con sus años o periodos
  urls <- list(
    '2023' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2023/defunciones_base_datos_2023_dbf.zip',
    '2022' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2022/defunciones_base_datos_2022_dbf.zip',
    '2021' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2021/defunciones_base_datos_2021_dbf.zip',
    '2020' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2020/defunciones_base_datos_2020_dbf.zip'
    # '2015_2019' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/datos/defunciones_generales_base_datos_2015_2019_dbf.zip'
  )
  
  for (periodo in names(urls)) {
    url <- urls[[periodo]]
    zip_path <- file.path(dir_datos, basename(url))
    download.file(url, zip_path, mode = "wb")
    
    # Carpeta para extraer
    unzip_dir <- file.path(dir_datos, paste0('Defunciones_', periodo))
    dir.create(unzip_dir, showWarnings = FALSE)
    
    unzip(zip_path, exdir = unzip_dir)
    unlink(zip_path)  # borrar zip
    
    # Buscar archivos .dbf en el unzip_dir
    dbf_files <- list.files(unzip_dir, pattern = '\\.dbf$', full.names = TRUE, ignore.case = TRUE)
    
    if(length(dbf_files) == 0) {
      warning(paste0('No se encontró archivo DBF para ', periodo))
      next
    }
    
    # Leer cada DBF y guardarlo en lista
    period_lista_defun <- list()
    for(dbf_file in dbf_files) {
      nombre <- file_path_sans_ext(basename(dbf_file))
      df <- tryCatch({
        read.dbf(dbf_file, as.is = TRUE) %>% as_tibble() %>% 
          clean_names()
      }, error = function(e) {
        warning(paste0('Error leyendo DBF: ', dbf_file))
        NULL
      })
      
      if(!is.null(df)) {
        period_lista_defun[[nombre]] <- df
      }
    }
    
    lista_defun[[periodo]] <- period_lista_defun
  }
  
  # Guardar la lista completa como archivo RDS
  saveRDS(lista_defun, file = 'Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')
  return(lista_defun)
}

In [26]:
defunciones <- descargar_defunciones_dbf()

Descargar los datos del Subsistema de Información sobre Nacimientos (SINAC) de la Secretaría de Salud Federal.

In [18]:
dir.create('SINAC')
options(timeout = 600) #Aumentar el tiempo de espera para la descarga
#Crear lista para guardar los datos de sinac
lista_nacimientos <- list()

Warning message in dir.create("SINAC"):
"'SINAC' already exists"


In [ ]:
#iterar por los años de descarga
    for (year in 19:23) {
#Crear la cadena de los años 2020 a 2023
        
        year_str <- paste0('20', year)
        #Cosntruir la url de descarga
        url <- paste0('http://www.dgis.salud.gob.mx/descargas/datosabiertos/nacimientos/sinac_', year_str, '.zip')
        
        #nombre y carpeta del archivo zip
        zip_file <- paste0('SINAC/sinac_', year_str, '.zip')
        extract_dir <- paste0('SINAC/sinac_', year_str)
#Crear carpeta si no existe
        if(!dir.exists('SINAC')) dir.create('SINAC')

#Intentar descargar archivos con trycatch
        tryCatch({
            download.file(url, destfile = zip_file, mode = 'wb')
            
            #Crear carpeta de extracción si no existe
            if(!file.exists(extract_dir)) dir.create(extract_dir)
                          
            #Descrompimir el archivo zip
            unzip(zip_file, exdir = extract_dir)

             #Buscar los archivos descomprimidos
            files <- list.files(extract_dir, full.names = TRUE)
    
    #Leer el primer archivo CSV encontrado (ajusta si es otro formato)
            csv_file <- files[grep("\\.csv$", files)][1]
    
        if (!is.na(csv_file)) {
        df <- read.csv(csv_file, stringsAsFactors = FALSE) %>% clean_names()
      
        # Guardar el dataframe en la lista
        lista_nacimientos[[year_str]] <- df
        
        # Eliminar el archivo ZIP
        file.remove(zip_file)

        #Guardar la lista completa para no volver a descargar los datos
        saveRDS(lista_nacimientos, file = 'SINAC/sinac_data.rds')
        
        message(paste('Año', year_str, 'procesado correctamente.'))
        } else {
        message(paste('No se encontró archivo CSV en', extract_dir))
        }
    }, error = function(e) {
        message(paste('Error al procesar el año', year_str))
    })
    }


Año 2019 procesado correctamente.

Año 2020 procesado correctamente.

Año 2021 procesado correctamente.

Año 2022 procesado correctamente.

Año 2023 procesado correctamente.



Descargar los datos del Censo Población y Vivienda 2020

In [53]:
#Carpeta donde se guardará todo
base_dir <- 'Censo Poblacion y Vivienda 2020'

#Url de descarga
url <- url <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/tabulados/cpv2020_b_cdmx_01_poblacion.xlsx'

# Ruta de guardado del archivo
excel_file <- file.path(base_dir, 'cpv2020_b_cdmx_01_poblacion.xlsx')


In [56]:
# Crear carpeta base si no existe
if (!dir.exists(base_dir)) dir.create(base_dir)

# Descargar el archivo Excel si no existe
if (!file.exists(excel_file)) {
  download.file(url, destfile = excel_file, mode = 'wb')
  message('Archivo Excel descargado correctamente.')
} else {
  message('El archivo Excel ya existe, no se descarga de nuevo.')
}

# Leer la hoja correspondiente
censo_tabla <- read_excel(excel_file, sheet = '03', range = 'A9:G1759') %>% 
clean_names()

colnames(censo_tabla) <- c('entidad_federativa', 'demarcacion_territorial', 'edad_desplegada', 'poblacion_total', 'hombres', 'mujeres', 'relacion_hombres_mujeres')

# Guardar el dataframe para no volver a cargarlo
saveRDS(censo_tabla, file = file.path(base_dir, 'censo_data.rds'))

message('Censo 2020 leído y guardado correctamente.')

El archivo Excel ya existe, no se descarga de nuevo.



New names:
• `Total` -> `Total...2`
• `Total` -> `Total...3`
Censo 2020 leído y guardado correctamente.



Datos de población CONAPO

In [ ]:
dir.create('CONAPO', showWarnings = FALSE)

#Carpeta donde se guardará todo
base_dir <- 'CONAPO'

#Url de descarga
url <- 'https://conapo.segob.gob.mx/work/models/CONAPO/pry23/DB/ConDem50a19_ProyPob20a70.zip'

#Archivo zip y carpeta de extracción
zip_file <- file.path(base_dir, 'conapo_data.zip')

extract_dir <- file.path(base_dir, '0_Pob_Mitad_1950_2070')

In [12]:
if(!file.exists(zip_file)) {
  download(url, destfile = zip_file, mode = 'wb')
}

if (!dir.exists(extract_dir)) {
  archive_extract(zip_file, dir = base_dir)
}

print(list.files(base_dir, recursive = TRUE))

files <- list.files(base_dir, pattern = '\\.xlsx$', full.names = TRUE, recursive = TRUE)

if(length(files) == 0) stop('No se encontró el archivo excel')

excel_file <- files[2]
print(paste('Archivo encontrado:', excel_file))

pob_conapo <- read_xlsx(excel_file) %>% 
clean_names()

# Guardar el dataframe para no volver a cargarlo
saveRDS(pob_conapo, file = file.path(base_dir, 'conapo_data.rds'))

message('Datos CONAPO leídos correctamente')


[1] "conapo_data.zip"                                                          
[2] "ConDem50a19_ProyPob20a70/0_Pob_Inicio_1950_2070.xlsx"                     
[3] "ConDem50a19_ProyPob20a70/0_Pob_Mitad_1950_2070.xlsx"                      
[4] "ConDem50a19_ProyPob20a70/1_Defunciones_1950_2070.xlsx"                    
[5] "ConDem50a19_ProyPob20a70/2_mig_inter_quinquen_proyecciones.xlsx"          
[6] "ConDem50a19_ProyPob20a70/3_mig_interest_quinquenal_proyecciones.xlsx"     
[7] "ConDem50a19_ProyPob20a70/4_Tasas_Especificas_Fecundidad_proyecciones.xlsx"
[8] "ConDem50a19_ProyPob20a70/5_Indicadores_demográficos_proyecciones.xlsx"    
[9] "ConDem50a19_ProyPob20a70/6_Esperanza_Vida_Nacer_1950_2070.xlsx"           
[1] "Archivo encontrado: CONAPO/ConDem50a19_ProyPob20a70/0_Pob_Mitad_1950_2070.xlsx"


Datos CONAPO leídos correctamente



Descargar datos ENANID

In [5]:
dir.create('ENADID')

Warning message in dir.create("ENADID"):
"'ENADID' already exists"


In [15]:
lista_enadid <- list()

In [16]:
descargar_enadid <- function() {

  options(timeout = 60000)

  enadid_urls <- list(
    '2023' = 'https://www.inegi.org.mx/contenidos/programas/enadid/2023/microdatos/base_datos_enadid23_csv.zip',
    '2018' = 'https://www.inegi.org.mx/contenidos/programas/enadid/2018/microdatos/base_datos_enadid18_csv.zip',
    '2014' = 'https://www.inegi.org.mx/contenidos/programas/enadid/2014/microdatos/base_datos_enadid14_dbf.zip'
  )

  dir_enadid <- 'ENADID'

  if(!dir.exists(dir_enadid)) dir.create(dir_enadid)
  lista_enadid <- list()
  
  for (anio in names(enadid_urls)) {
    url <- enadid_urls[[anio]]
    zip_file <- file.path(dir_enadid, basename(url))
    extract_dir <- file.path(dir_enadid, paste0('ENADID_', anio))
    if(!dir.exists(extract_dir)) dir.create(extract_dir)
    download.file(url, destfile = zip_file, mode = 'wb')
    unzip(zip_file, exdir = extract_dir)
    file.remove(zip_file)
    
    # Buscar archivos csv o dbf
    csvs <- list.files(extract_dir, pattern = '\\.csv$', full.names = TRUE, ignore.case = TRUE)
    dbfs <- list.files(extract_dir, pattern = '\\.dbf$', full.names = TRUE, ignore.case = TRUE)
    
    if(length(csvs) > 0) {
      # Leer todos los CSV y unirlos si hay más de uno
      df <- lapply(csvs, function(f) read.csv(f, stringsAsFactors = FALSE) %>% clean_names())
     
     
      if(length(df) == 1) df <- df[[1]] else df <- bind_rows(df)
      lista_enadid[[anio]] <- df
    } else if(length(dbfs) > 0) {
      
      
      # Leer todos los DBF y unirlos si hay más de uno
      df <- lapply(dbfs, function(f) read.dbf(f, as.is = TRUE) %>% as_tibble() %>% clean_names())
      if(length(df) == 1) df <- df[[1]] else df <- bind_rows(df)
      lista_enadid[[anio]] <- df
    } else {
      warning(paste('No se encontraron archivos CSV o DBF para ENADID', anio))
    }
  }
  
  saveRDS(lista_enadid, file = file.path(dir_enadid, 'enadid_data.rds'))
  
  return(lista_enadid)
}

In [17]:
invisible(descargar_enadid())

Cargar todos los datos previamente descargados

In [166]:
#Función para cargar todos los datos descargados
datos_demo <- function(){
  list(
    lista_nacimientos = readRDS('SINAC/sinac_data.rds'),
    lista_defun = readRDS('Estadísticas de Defunciones Registradas (EDR)/edr_data.rds'),
    censo_tabla = readRDS('Censo Poblacion y Vivienda 2020/censo_data.rds'),
    pob_conapo = readRDS('CONAPO/conapo_data.rds'),
    lista_enadid = readRDS(('ENADID/enadid_data.rds')))
}

# Cargar los datos en un objeto
demo <- datos_demo()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Cálculo de Indicadores

In [122]:
#Años que nos van a servir para iterar
anios <- 2020:2023

Tasa de embarazo adolescente

(Embarazos de mujeres 15-19 / Total de mujeres de 15-19 años ) * 1000

In [5]:
madres_adole <- tibble(anio = integer(), nacimientos = integer())

In [6]:
for (a in anios) {
    anio_str <- as.character(a)

    if (!is.null(demo$lista_nacimientos[[anio_str]])) {
        df_nac <- demo$lista_nacimientos[[anio_str]] %>% as_tibble()
        
        # Definir las variables Entidad y Edad por año
        if (a == 2019) {
            var_entidad <- 'ent_cert'
            var_edad <- 'edadm'
        } else {
            var_entidad <- 'entidadfederativacertifica'
            var_edad <- 'edad'
        }
        
        # Filtrar y calcular
        nacimientos <- df_nac %>%
            filter(.data[[var_entidad]] == 9,
            .data[[var_edad]] >= 15 & .data[[var_edad]] <= 19) %>%
            summarise(nacimientos = n()) %>%
            pull(nacimientos)
        
        # Guardar los resultados en la tabla creada
        madres_adole <- madres_adole %>%
        add_row(anio = a, nacimientos = nacimientos)
        
    } else {
        message(paste('Datos incompletos para el año', a))
    }
}

# Ver resultados
print(madres_adole)

Datos incompletos para el año 2014

Datos incompletos para el año 2015

Datos incompletos para el año 2016

Datos incompletos para el año 2017

Datos incompletos para el año 2018



# A tibble: 5 × 2
   anio nacimientos
  <int>       <int>
1  2019       15042
2  2020       12991
3  2021       10542
4  2022       10112
5  2023        9660


In [9]:
adole <- as_tibble(demo$censo_tabla) %>%
filter(edad_desplegada %in% c('15 años', '16 años', '17 años', '18 años', '19 años'),
demarcacion_territorial == 'Total') %>%
summarise(adole = sum(mujeres)) %>%
pull(adole)

adole

[1] 319280

In [10]:
madres_adole <- madres_adole %>%
mutate(tasa_fecun = (nacimientos / adole) * 1000)

madres_adole

anio,nacimientos,tasa_fecun
<int>,<int>,<dbl>
2019,15042,47.11225
2020,12991,40.68842
2021,10542,33.01804
2022,10112,31.67126
2023,9660,30.25558


In [13]:
write_xlsx(madres_adole, 'Resultados/Tasa de embarazo adolescente.xlsx')

Tasa Bruta de Mortalidad

In [155]:
#Conjunto de nombres para los dataframes de defunciones
nombres_def <- c('2020' = 'defun20','2021' = 'defun21', '2022' = 'DEFUN22', '2023' = 'DEFUN23')

In [154]:
rm(tbm_df)

In [156]:
tbm_df <- tibble(anio = integer(), defunciones = integer(), poblacion = numeric(), tasa = numeric())

In [157]:
# Iterar sobre los años para calcular tasas de defunciones por 1000 habitantes
for (a in anios) {
  anio_str <- as.character(a)
  
  # Validar que existan datos para el año actual
  if (!is.null(demo$lista_defun[[anio_str]]) && !is.null(demo$pob_conapo)) {
    
    nombre_df <- nombres_def[anio_str]
    nombre_df <- unname(nombre_df)
    
    df_def <- demo$lista_defun[[anio_str]][[nombre_df]]
    df_pob <- as_tibble(demo$pob_conapo)
    
    # Calcular defunciones totales para CDMX
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a) %>%
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Filtrar población total del año
    pob <- df_pob %>%
    filter(entidad == 'Ciudad de México', anio == a) %>%
    summarise(poblacion_total = sum(poblacion)) %>%
    pull(poblacion_total)


    tasa <- (def / pob) * 1000
    
    # Guardar resultados
    tbm_df <- tbm_df %>%
      add_row(anio = a, defunciones = def, poblacion = pob, tasa = tasa)
    
  } else {
    message(paste('Datos incompletos para el año', a))
    tbm_df <- tbm_df %>% add_row(anio = a, defunciones = NA, poblacion = NA, tasa = NA)
  }
}

print(tbm_df)


# A tibble: 4 × 4
   anio defunciones poblacion  tasa
  <int>       <int>     <dbl> <dbl>
1  2020      106968         0   Inf
2  2021       99785         0   Inf
3  2022       69095         0   Inf
4  2023       65051         0   Inf


Tasa de Mortalidad infantil Ciudad de México

In [14]:
#Conjunto de nombres para los dataframes de defunciones
nombres_def <- c('2020' = 'defun20','2021' = 'defun21', '2022' = 'DEFUN22', '2023' = 'DEFUN23')

   Menores de 5 años

In [15]:
#Crear dataframe vacío para almacenar los resultados
mortalidad_cinco <- tibble(anio = integer(), nacimientos = integer(), defunciones = integer(), tasa = numeric())


In [16]:
# Iterar sobre los años y calcular tasas
for (a in anios) {
  anio_str <- as.character(a)
  
  # Validar que existan datos para el año actual
  if (!is.null(demo$lista_nacimientos[[anio_str]]) && !is.null(demo$lista_defun[[anio_str]])) {
    
    # Extraer el dataframe de nacimientos
    df_nac <- demo$lista_nacimientos[[anio_str]]
    
    # Obtener el nombre correcto del dataframe de defunciones
    nombre_df <- nombres_def[anio_str]
    df_def <- demo$lista_defun[[anio_str]][[nombre_df]]
    
    # Calcular nacimientos para CDMX
    nac <- df_nac %>%
      filter(entidadresidencia == 9) %>%
      summarise(nacimientos = n()) %>%
      pull(nacimientos)
    
    # Calcular defunciones infantiles para CDMX
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a,
      ((edad >= 1001 & edad <= 1098) |  #Horas y minutos
      (edad >= 2001 & edad <= 2998) |  #Días
      (edad >= 3001 & edad <= 3098) |  #Meses
      (edad >= 4001 & edad <= 4004))) %>%    #Años cumplidos (1 a 4)
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Calcular la tasa
    tasa <- (def / nac) * 1000
    
    # Guardar resultados
    mortalidad_cinco <- mortalidad_cinco %>% 
      add_row(anio = a, nacimientos = nac, defunciones = def, tasa = tasa)
    
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

# Mostrar resultados
print(mortalidad_cinco)


Datos incompletos para el año 2014

Datos incompletos para el año 2015

Datos incompletos para el año 2016

Datos incompletos para el año 2017

Datos incompletos para el año 2018

Datos incompletos para el año 2019



# A tibble: 4 × 4
   anio nacimientos defunciones  tasa
  <int>       <int>       <int> <dbl>
1  2020       87005        1266  14.6
2  2021       77476        1062  13.7
3  2022       80892        1159  14.3
4  2023       78032        1152  14.8


In [20]:
write_xlsx(mortalidad_cinco, 'Resultados/Tasa de mortalidad 5 años.xlsx')

Menores de 1 año

In [45]:
#Crear dataframe vacío para almacenar los resultados
mortalidad_uno <- tibble(anio = integer(), nacimientos = integer(), defunciones = integer(), tasa = numeric())


In [46]:
# Iterar sobre los años y calcular tasas
for (a in anios) {
  anio_str <- as.character(a)
  
  # Validar que existan datos para el año actual
  if (!is.null(demo$lista_nacimientos[[anio_str]]) && !is.null(demo$lista_defun[[anio_str]])) {
    
    # Extraer el dataframe de nacimientos
    df_nac <- demo$lista_nacimientos[[anio_str]]
    
    # Obtener el nombre correcto del dataframe de defunciones
    nombre_df <- nombres_def[anio_str]
    df_def <- demo$lista_defun[[anio_str]][[nombre_df]]
    
    # Calcular nacimientos para CDMX
    nac <- df_nac %>%
      filter(entidadresidencia == 9) %>%
      summarise(nacimientos = n()) %>%
      pull(nacimientos)
    
    # Calcular defunciones infantiles para CDMX
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a, 
      ((edad >= 1001 & edad <=  1098) | #Horas y minutos
      (edad >= 2001 & edad <= 2998) | #Días
      (edad >=  3001 & edad <= 3098))) %>% #Meses
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Calcular la tasa
    tasa <- (def / nac) * 1000
    
    # Guardar resultados
    mortalidad_uno <- mortalidad_uno %>% 
      add_row(anio = a, nacimientos = nac, defunciones = def, tasa = tasa)
    
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

# Mostrar resultados
print(mortalidad_uno)


Datos incompletos para el año 2014

Datos incompletos para el año 2015

Datos incompletos para el año 2016

Datos incompletos para el año 2017

Datos incompletos para el año 2018

Datos incompletos para el año 2019



# A tibble: 4 × 4
   anio nacimientos defunciones  tasa
  <int>       <int>       <int> <dbl>
1  2020       87005        1104  12.7
2  2021       77476         927  12.0
3  2022       80892        1020  12.6
4  2023       78032        1014  13.0


In [47]:
write_xlsx(mortalidad_uno, 'Resultados/Tasa de mortalidad 1 año.xlsx')

Razón de dependencia demográfica

Cociente de (pob menores a 15 + mayores de 64 )/ pob 15 - 64 'edad productiva'

In [23]:
pob_conapo <- as_tibble(demo$pob_conapo)
print(pob_conapo)

# A tibble: 737,660 × 7
   renglon   ano entidad            cve_geo  edad sexo    poblacion
     <dbl> <dbl> <chr>                <dbl> <dbl> <chr>       <dbl>
 1       1  1950 República Mexicana       0     0 Hombres    580871
 2       2  1950 República Mexicana       0     0 Mujeres    568159
 3       3  1950 República Mexicana       0     1 Hombres    525478
 4       4  1950 República Mexicana       0     1 Mujeres    516674
 5       5  1950 República Mexicana       0     2 Hombres    492568
 6       6  1950 República Mexicana       0     2 Mujeres    483825
 7       7  1950 República Mexicana       0     3 Hombres    469166
 8       8  1950 República Mexicana       0     3 Mujeres    460137
 9       9  1950 República Mexicana       0     4 Hombres    450189
10      10  1950 República Mexicana       0     4 Mujeres    440962
# ℹ 737,650 more rows


In [85]:
#Años para iterar en datos CONAPO
years <- seq(2010, 2045, by = 5)

In [87]:
depen_demo <- tibble(ano = integer(), rd = integer())

In [88]:
for (year in years) {
    
    pob_conapo <- as_tibble(demo$pob_conapo)

    #Población menor de 15 (dependiente)
    pob_men_15 <- pob_conapo %>%
    filter(entidad == 'Ciudad de México', ano == year, edad < 15) %>%
    summarise(pob_men_15 = sum(poblacion)) %>%
    pull(pob_men_15)

    #Población mayor de 64 (dependiente)
    pob_mayor_64 <- pob_conapo %>%
    filter(entidad == 'Ciudad de México', ano == year, edad > 64) %>%
    summarise(pob_mayor_64 = sum(poblacion)) %>%
    pull(pob_mayor_64)

    #Población con edad productiva
    pob_edad_produc <- pob_conapo %>%
    filter(entidad == 'Ciudad de México', ano == year, edad %in% c(15:64)) %>%
    summarise(pob_edad_produc = sum(poblacion)) %>%
    pull(pob_edad_produc)

#Razón de dependencia demográfica
rd <- ((pob_men_15 + pob_mayor_64) / pob_edad_produc) * 100

#Guardar resultados
depen_demo <- depen_demo %>%
add_row(ano = year, rd = rd)

}

print(depen_demo)

# A tibble: 8 × 2
    ano    rd
  <dbl> <dbl>
1  2010  45.5
2  2015  43.2
3  2020  41.0
4  2025  40.1
5  2030  41.1
6  2035  44.2
7  2040  48.1
8  2045  51.7


In [28]:
write_xlsx(depen_demo, 'Resultados/Razón de dependencia demográfica.xlsx')

Índice de envejecimiento (CONAPO)

In [89]:
indice_vej <- tibble(anio = integer(), indice = numeric())

In [90]:
for (year in years) {
    
    pob_conapo <- as_tibble(demo$pob_conapo)

    #Población menor de 15 
    pob_men_14 <- pob_conapo %>%
    filter(entidad == 'Ciudad de México', ano == year, edad <= 14) %>%
    summarise(pob_men_14 = sum(poblacion)) %>%
    pull(pob_men_14)

    #Población mayor de 64 
    pob_mayor_60 <- pob_conapo %>%
    filter(entidad == 'Ciudad de México', ano == year, edad >= 60) %>%
    summarise(pob_mayor_60 = sum(poblacion)) %>%
    pull(pob_mayor_60)


#Índice de envejecimiento
indice_actual <- (pob_mayor_60 / pob_men_14) * 100 %>%
round(1)


#Guardar resultados
indice_vej <- indice_vej %>%
add_row(anio = year, indice = indice_actual)

}

print(indice_vej)

# A tibble: 8 × 2
   anio indice
  <dbl>  <dbl>
1  2010   49.2
2  2015   64.9
3  2020   82.0
4  2025  111. 
5  2030  150. 
6  2035  192. 
7  2040  233. 
8  2045  283. 


In [211]:
write_xlsx(indice_vej, 'Resultados/Índice de envejecimiento.xlsx')

Tasa Bruta de Mortalidad

In [202]:
defunciones_total <- list()

In [203]:
for (anio in anios) {
  # Extraer la sublista del año correspondiente
  sublista <- demo$lista_defun[[as.character(anio)]]
  
  # Buscar el nombre de la tabla que contiene "defun" (insensible a mayúsculas)
  nombre_tabla <- names(sublista)[str_detect(names(sublista), regex('DEFUN', ignore_case = TRUE))]
  
  if (length(nombre_tabla) > 0) {
    # Extraer la primera tabla encontrada
    tabla <- sublista[[nombre_tabla[1]]]
    
    # Filtrar defunciones de residentes en CDMX
    tabla_filtrada <- tabla %>%
      filter(ent_resid == "09")
    
    # Guardar el total de defunciones del año
    defunciones_total[[as.character(anio)]] <- nrow(tabla_filtrada)
  } else {
    # Si no se encuentra tabla, asignar NA
    defunciones_total[[as.character(anio)]] <- NA
  }
}

print(defunciones_total)

$`2020`
[1] 107292

$`2021`
[1] 100363

$`2022`
[1] 69455

$`2023`
[1] 65291



In [204]:
#Convertir la lista a una tabla
defunciones <- tibble(anio = as.integer(names(defunciones_total)),
            defunciones = as.numeric(unlist(defunciones_total)))
print(defunciones)

# A tibble: 4 × 2
   anio defunciones
  <int>       <dbl>
1  2020      107292
2  2021      100363
3  2022       69455
4  2023       65291


In [206]:
# Obtener población de CDMX por año desde CONAPO
poblacion_cdmx <- demo$pob_conapo %>%
rename(anio = ano) %>%
filter(entidad == "Ciudad de México", anio >= 2020, anio <= 2023) %>%
group_by(anio) %>%
summarise(poblacion_cdmx = sum(poblacion))

poblacion_cdmx

anio,poblacion_cdmx
<dbl>,<dbl>
2020,9332593
2021,9272649
2022,9237644
2023,9221637


In [209]:
tasa <- defunciones %>%
left_join(poblacion_cdmx, by = 'anio') %>%
mutate(tasa = (defunciones / poblacion_cdmx) * 1000)

tasa

anio,defunciones,poblacion_cdmx,tasa
<dbl>,<dbl>,<dbl>,<dbl>
2020,107292,9332593,11.496483
2021,100363,9272649,10.823552
2022,69455,9237644,7.518692
2023,65291,9221637,7.080196


In [210]:
write_xlsx(tasa, 'Resultados/Tasa bruta de defunciones.xlsx')